This is a notes of Kitaev spin liquid under a local magnetic field. The code was written in real space with maximum $80 \times 80 \times 2$ sites. 

\section{Hamiltonian}

\begin{align}
    H &= H_K + H_{h} \\
    H_K &= - \sum_{<j,k>_\alpha} J_\alpha \sigma_j^\alpha \sigma_k^\alpha \\
    H_h &= - \sum_\alpha h_\alpha \sigma_0^\alpha
\end{align}

The ground state Hamiltonain of Kitaev model is :
\begin{equation}
    H_K^0 = \sum_{<j,k>_\alpha} J_\alpha \ i c_j c_k
\end{equation}

When applied the local magnetic field, the $<u_{<0j>_\alpha}$ is not conserved anymore while others are still conserved quanties. We can rewrite the Hamiltonian into this form:

\begin{align}
\begin{split}
    H &= H_K^0 + \sum_{<0j>_\alpha} J_\alpha \ (i b_0^\alpha b_j^\alpha - 1) \ i c_0 c_j
            - \sum_\alpha h_\alpha \ i c_0 b_0^\alpha \\
      & = H_K^0 + H_{int} + H_h\\
      & = H_K^0 + H_{imp}
\end{split}
\end{align}

where $H_{imp} = H_{int} + H_h$.


\section{MFT Hamiltonian}

Apply MFT to $H_{int}$, we have:
\begin{equation}
    H_{int}^{MF} = \sum_\alpha J_\alpha ( <i c_0 c_\alpha> i b_0^\alpha b_\alpha^\alpha  
                        + <i b_0^\alpha b_\alpha^\alpha> i c_0 c_\alpha
                        - <i c_0 b_0^\alpha> i c_\alpha b_\alpha^\alpha
                        - <i c_\alpha b_\alpha^\alpha> i c_0 b_0^\alpha)
\end{equation}
The constant term is:
\begin{equation}
    H_{int}^{C} = \sum_\alpha J_\alpha( <i c_0 b_0^\alpha> <i c_\alpha b_\alpha^\alpha>
                                    - <i c_0 c_\alpha><i b_0^\alpha b_\alpha^\alpha>)
\end{equation}

where we use superscript $\alpha$ to label bonds direction and subscript $\alpha$ to label atoms link to the $0$-atom with bond $\alpha$. And we ignore the $(i c_0 b_\alpha^\alpha)(i b_0^\alpha c_\alpha)$ terms because they have been proved $0$ in previous studies.

There should be a local constrain on site-$0$: $i c_0 b_0^x b_0^y b_0^z =1$. It is applied as Lagrange multipliers:
\begin{equation}
    H_{lag} = \sum_\alpha \lambda_\alpha ( i c_0 b_0^\alpha 
                    + \epsilon_{\alpha \beta \gamma} i b_0^\beta b_0^\gamma)
\end{equation}

\section{Lattice}

We use a $N_1 \times N_2 \times 2$ rhomboid lattice. The lattice vectors are $\vec{n}_1 = (0,1), \vec{n}_2 = (\frac{1}{2},\frac{\sqrt{3}}{2})$. The number of unit cells along these two directions are $N_1$ and $N_2$ respectively. We label each unit cell by its row and colume numbers $(r, c)$(start from $0$), the label of this unit cell given by $ n = r \times N_1 + c$. We can label A atom in the $n$-th unit cell as $2n$ and B atom as $2n + 1$.

For a given A atom $2n$, the B atom linked to it by $x$-bond is $2n + 1$, $y$-bond is $2n - 1$, $z$-bond is $2 (n - N_1) + 1$.

we use the basis $(c_{1A},\ c_{1B},\ ... \,\ c_{NA},\ c_{NB})^T$ when there is no magnetic field. When $H_h$ is added, the basis dimension becomes $N_1 \times N_2 \times 2 + 6$,  where the last six operators in the basis is $(b_0^x,\ b_0^y,\ b_0^z,\ b_x^x,\ b_y^y,\ b_z^z)$.

In [1]:
import numpy as np
from scipy.sparse import csc_matrix

def BulkHam(JK, N1,N2, periodic = False, flux = False):
    '''Kitaev Hamiltonian in fixed flux pattern
    
       Inputs: JK : 1*3 matrix, Jx=J[0], Jy = J[1], Jz=J[2]
               N1,N2: int
               
       Return: H matrix'''
    
    SiteNum = N1 * N2 * 2
    Hr = np.zeros([SiteNum, SiteNum],dtype=complex)
    for r in range(N2):
        for c in range(N1):
            n = r * N1 + c #label of unit cell
            '''X-bonds'''
            next = 2 * n + 1
            Hr[2*n, next] +=  1j * JK[0]  
                
            '''Y-bonds'''
            if c > 0:
                next = 2*n -1
            elif c == 0 and periodic == True:
                next = 2 * (n + N1) -1
            else:
                next = 0.5
                    
            if next % 1 == 0:
                Hr[2*n, next] += 1j * JK[1]
   
            '''Z-bonds'''
            if r > 0:
                next = 2*(n - N1) + 1
            elif r == 0 and periodic == True:
                next = 2 * (n + N1 * N2 - N1) +1
            else:
                next = 0.5
                
            if next % 1 == 0:
                if flux == True and r == N2//2 and c >= N1//2:
                    Hr[2*n, next] +=  -1j * JK[2]
                else:
                    Hr[2*n, next] +=  1j * JK[2]
                    
    Hr = (Hr + np.conj(Hr).T)/2
    return Hr

def BulkHam_csc(JK, N1,N2, periodic = False, flux = False):
    '''Kitaev Hamiltonian in fixed flux pattern
    
       Inputs: JK : 1*3 matrix, Jx=J[0], Jy = J[1], Jz=J[2]
               N1,N2: int
               
       Return: positions and values of non-zero elements of HK^0'''
    
    SiteNum = N1 * N2 * 2
    
    row = []
    col = []
    data = []
    for r in range(N2):
        temp = {} # A temporary dict
        for c in range(N1):
            n = r * N1 + c #label of unit cell
            '''X-bonds'''
            next = 2*n + 1
            if next in temp:
                temp[next] += 1j * JK[0]
            else: 
                temp[next] = 1j * JK[0]
                
            '''Y-bonds'''
            if c > 0:
                next = 2*n -1
            elif c == 0 and periodic == True:
                next = 2 * (n + N1) -1
            else:
                next = 0.5
                    
            if next % 1 == 0:
                if next in temp:
                    temp[next] += 1j * JK[1]
                else: 
                    temp[next] = 1j * JK[1]
   
            '''Z-bonds'''
            if r > 0:
                next = 2*(n - N1) + 1
            elif r == 0 and periodic == True:
                next = 2 * (n + N1 * N2 - N1) +1
            else:
                next = 0.5
                
            if next % 1 == 0:
                if next in temp:
                    if flux == True and r == N2//2 and c >= N1//2:
                        temp[next] +=  -1j * JK[2]
                    else:
                        temp[next] +=  1j * JK[2]
                else: 
                    if flux == True and r == N2//2 and c >= N1//2:
                        temp[next] =  -1j * JK[2]
                    else:
                        temp[next] =  1j * JK[2]
            odTemp = sorted(temp)
            for e in odTemp:
                row.append(2*n)
                col.append(e)
                data.append(temp[e])
                    
    return row, col, data

def ImpHam_csc(JK, N1,N2, h, la, m0, ma, cc, ua):
    '''H_imp = H_int + H_h
    
       Inputs: h:1*3 matrix, hx=h[0], hy = h[1], hz=h[2]
               la : 1*3 matrix, Lagrange multipliers
               m0: 1*3 matrix, i c0 b0^a
               m0: 1*3 matrix, i ca ba^a
               cc: 1*3 matrix, i c0 ca
               ua: 1*3 matrix, i b0^a ba^a
               
       Return: positions and values of non-zero elements of H_imp'''
    imp = N1//2 * N1 + N2//2 # position of the impurity site
    next = np.array([2*imp +1, 2*imp -1, 2*(imp - N1) +1]) # positions of atoms linked to the impurity
    
    SiteNum = N1 * N2 * 2
    row = []
    col = []
    data = []
    
    '''Effective magnetic field'''
    for i in range(3):
        row.append( 2*imp )
        col.append( SiteNum + i )
        data.append( 1j * h[i] + 1j * la[i])
    
    '''Second term of the lagrange multipliers'''
    for i in range(3):
        row.append( SiteNum + (i+1)%3 )
        col.append( SiteNum + (i+2)%3 )
        data.append( 1j * la[i])
    
    '''H_int : i c0 ca'''
    for i in range(3):
        row.append( 2*imp )
        col.append( next[i] )
        data.append( 1j * ua[i] )
        
    '''H_int : i b0^a ba^a'''
    for i in range(3):
        row.append( SiteNum + i )
        col.append( SiteNum + 3 + i )
        data.append( 1j * cc[i] )
        
    '''H_int : i c0 b0^a'''
    for i in range(3):
        row.append( 2*imp )
        col.append( SiteNum + i )
        data.append( - 1j * ma[i] )

    '''H_int : i ca ba^a'''
    for i in range(3):
        row.append( next[i] )
        col.append( SiteNum + 3 + i )
        data.append( - 1j * m0[i] )  
    return row, col, data

def makeH(row, col, data, dim):
    '''Make csc format Hamiltonian
       
       Inputs : row,col :position of the non-zero elementds of Hamiltonian matrix
                data: H(row,col) = data
                dim: Hamiltonian matrix dimension
       Return: Hcsc: csc formate Hamiltonian
       
       Notice: we only have to enter half of the elements and get the other half 
               by hermition conjugate.'''
    Hcsc = csc_matrix((data, (row, col)), shape=(dim, dim),dtype = complex)
    Hcsc = (Hcsc + Hcsc.getH()) /2 
    return Hcsc


\section{Green's Functions}
\quad Once we have the Hamiltonian matrix, we proceed to calculate the order parameters. Suppose we already know the eigen values $\epsilon_j$ and corresponding eigen vectors $| \psi_j \rangle$. The Green's function is:
\begin{align}
\begin{split}
    \hat{G}(i \omega_n ) &=  \frac{1}{ i \omega_n - \hat{H}} \\
           &=  \frac{1}{ i \omega_n - \hat{H}} \sum_j | \psi_j \rangle \langle \psi_j |\\
           & =\sum_j \frac{| \psi_j \rangle \langle \psi_j |}{i \omega_n - \epsilon_j}
\end{split}
\end{align}

\begin{align}
\begin{split}
    \hat{G}(0^- ) &=  \frac{1}{ \beta } \sum_{i \omega_n} \hat{G}(i \omega_n ) 
                       e^{ i \omega_n 0^-}\\
           &=  - \int_C \frac{\rm{d} z}{2 \pi i } f(z) G(z) e^{z 0^-}\\
           & =\sum_j | \psi_j \rangle \langle \psi_j | f( \epsilon_j)
\end{split}
\end{align}

where $f(\epsilon)$ is the Fermi distribution. Then we have:

\begin{equation}
    [\hat{G}(0^-]_{mn} = \sum_j | \psi_j\rangle_m \langle \psi_j |_n f(\epsilon_l)
\end{equation}

\quad For Majorana fermions, there is an extra factor $2$, we modify $f(\epsilon)$ to $2 f(\epsilon)$.

In [118]:
'''PLEASE RUN THE 1st CODE SECTOR BEFORE THIS ONE.'''
def Fermi(eng, T):
    '''Fermi distribution function
       Inputs: eng : Energy
               T : Temperature '''
    fermi = 1 / (1 + np.exp(eng / T) )
    return fermi

def GreensFun(N1, N2, val,vec):
    '''Calculate the Green's functions of orderparameters.
       
       Inputs:N1, N2 : number of unit cells in two directions
              vals, vecs: eigen values and corresponding eigen vectors of the 
                           Hamiltonian
               
               
       Return: bb, m0, ma, cc, ua : parameters that need to be calculated
               bb is the second term of lagrange multiplier'''
    
    bb = np.zeros(3,dtype = complex)
    m0New = np.zeros(3,dtype = complex)
    maNew = np.zeros(3,dtype = complex)
    ccNew = np.zeros(3,dtype = complex)
    uaNew = np.zeros(3,dtype = complex)
    
    imp = N1//2 * N1 + N2//2 # position of the impurity site
    next = np.array([2*imp +1, 2*imp -1, 2*(imp - N1) +1]) # positions of atoms linked to the impurity
    SiteNum = N1 * N2 * 2
    
    Num = len(val)
    
    for l in range(Num//2):
        fermi = 2* Fermi(val[l], T)
        for i in range(3):
            '''i c0 ca'''
            ccNew[i] += 1j * np.conj(vec[2* imp, l]) * vec[next[i], l] * fermi
            '''i b0^a ba^a'''
            uaNew[i] += 1j * np.conj(vec[SiteNum + i,l]) * vec[SiteNum + 3 + i, l] * fermi
            '''i c0 b0^a'''
            m0New[i] += 1j * np.conj(vec[2* imp, l]) * vec[SiteNum + i, l] * fermi
            '''i ca ba^a'''
            maNew[i] += 1j * np.conj(vec[next[i], l]) * vec[SiteNum + 3 + i,l] * fermi
            '''Second term of the lagrange multipliers'''
            bb[i] += 1j * np.conj(vec[SiteNum + (i+1)%3,l]) * vec[SiteNum + (i+2)%3,l] * fermi
    return m0New, maNew, ccNew, uaNew, bb

def update(error, dt, la, m0, ma, cc, ua, m0New, maNew, ccNew, uaNew, bb):
    '''update order parameters
    
       Inputes: error: initiate error, default 1
                dt : time step
                la, m0, ma, cc, ua: order parameter need to be updated
                m0New, maNew, ccNew, uaNew, bb: new parameters
                
       Returm: error: finial error
               la, m0, ma, cc, ua: updated order parameters'''
    error = 0
    for i in range(3):
        delta = m0New[i] - m0[i]
        m0[i] += dt * delta
        error = max(error, abs(delta))
 
        delta = maNew[i] - ma[i]
        ma[i] += dt * delta
        error = max(error, abs(delta))
        
        delta = ccNew[i] - cc[i]
        cc[i] += dt * delta
        error = max(error, abs(delta))   
        
        delta = uaNew[i] - ua[i]
        ua[i] += dt * delta
        error = max(error, abs(delta))
        
        delta = m0New[i] + bb[i]
        la[i] += dt * delta
        error = max(error, abs(delta))
    return error, la, m0, ma, cc, ua

In [134]:
# System setups
JK = np.array([1,1,1])

N1 = 10
N2 = 10
SiteNum = N1 * N2 * 2
dim = SiteNum + 6

tolerance = 1.e-6
T = 1.e-5

# Initiate order parameters
m0 = np.array([0,0,0] ,dtype = complex)
ma = np.array([0,0,0] ,dtype = complex)
cc = np.array([-0.5284,-0.5284,-0.5284] ,dtype = complex)
ua = np.array([1,1,1] ,dtype = complex)
h = np.array([0,0,0] ,dtype = complex)
la = np.array([0,0,0] ,dtype = complex)

# Generate bulk Hamiltonian
row = []
col = []
data = []
row, col, data = BulkHam_csc(JK, N1,N2, periodic = True, flux = False)

# MFT process 
error = 1
icount = 0 # count the number of iterations

while (error > tolerance):
    icount += 1
    
    # Generate impurity Hamiltonian
    rowImp = []
    colImp = []
    dataImp = []
    rowImp, colImp, dataImp = ImpHam_csc(JK, N1,N2, h,la, m0, ma, cc, ua)
    
    # Make Hamiltonian matrix
    hcsc = makeH(row, col, data, dim) + makeH(rowImp, colImp, dataImp, dim) 
    harr = hcsc.toarray()
    
    val, vec = np.linalg.eigh(harr)
    
    m0New, maNew, ccNew, uaNew, bb = GreensFun(N1, N2, val,vec)
    
    dt = 0.05 + (1 - 2 * np.arctan(error * 100) / np.pi) * 0.04
    error, la, m0, ma, cc, ua = update(error, dt, la, m0, ma, cc, ua, m0New, maNew, ccNew, uaNew, bb)

print(icount)
print(error)
print('ua = ', ua  )
print('cc = ', cc  )
print('la=', la)

162
9.729637169553218e-07
ua =  [1.-4.66749783e-16j 1.+1.43808528e-16j 1.+9.83825409e-17j]
cc =  [-0.2288208 -1.18796084e-16j -0.56846642+5.83889826e-17j
 -0.51553771+3.82558210e-17j]
la= [-2.02400990e-17-1.03900056e-10j -1.56120862e-18+2.05396756e-09j
 -1.27578501e-17-3.16946925e-09j]
